In [44]:
import pandas as pd
import numpy as np
import re
from datetime import timedelta
import pycountry_convert as pc

In [45]:
df = pd.read_csv('main.csv', sep=';', encoding='windows-1251')

In [46]:
# shorten column names so that only data codes remain

df = df.rename(columns=lambda x: x.split(" ")[0])

In [47]:
# filter only necessary columns

df = df[['ND', 'G071', 'G072', 'GD1', 'G011', 'G141', 'G142', 'G143', 'G15', 'G15A', 'G202', 'G30CITY', 'G541_INN', 'G33', 'FIRM']]

In [48]:
# G33 (customs code) column values had a filter-like format for some reason

df['G33'] = df['G33'].str.replace('=', '')
df['G33'] = df['G33'].str.replace('"', '')

In [49]:
# extract the precise declaration number from the complex format used in 'ND' column
    
df['ND'] = df.ND.apply(lambda x: x.split("/")[2])

In [50]:
# drop duplicates to get unique declaration numbers

df = df.drop_duplicates().reset_index(drop=True)

In [51]:
# drop quotation marks and shorten legal entity form

dict_of_legal_entities = {'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ': 'ООО',
                            'ОТКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО': 'ОАО',
                            'АКЦИОНЕРНОЕ ОБЩЕСТВО': 'АО',
                            'ПУБЛИЧНОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО': 'ПАО',
                            'ЗАКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО': 'ЗАО',
                            'ИНДИВИДУАЛЬНЫЙ ПРЕДПРИНИМАТЕЛЬ': 'ИП',
                            '"': '',
                            '<': '',
                            '>': '',
                            '\'\'': ''}


def shorten_legal_entity(x):
    for key, value in dict_of_legal_entities.items():
        x = x.replace(key, value)
    return x


df['G142'] = df.G142.apply(shorten_legal_entity)

In [52]:
# place legal entity form after the company name

def reformat_company_name(x):
    if x.startswith(('ООО', 'ОАО', 'АО ', 'ПАО', 'ЗАО', 'ИП ')):
        return x[3:].strip() + ', ' + x[:3].strip()
    else:
        return x.strip()


df['G142'] = df.G142.apply(reformat_company_name)

In [53]:
# slight clarification of acronyms

df['G011'] = df.G011.apply(lambda x: x.replace('ИМ', 'Импорт'))
df['G011'] = df.G011.apply(lambda x: x.replace('ЭК', 'Экспорт'))

In [54]:
# convert dates to respective format and indicate shipments with late release (>1 days after registration) in a new column

df['G072'] = df.G072.apply(pd.to_datetime, format='%d.%m.%Y')
df['GD1'] = df.GD1.apply(pd.to_datetime, format='%d.%m.%Y')

df['late_release'] = df['GD1'] - df['G072'] > timedelta(1)

In [55]:
# extract registration months for aggregation

df['month_year'] = df['G072'].dt.month.astype(str) + '_' + df['G072'].dt.year.astype(str)

In [56]:
# total declarations per company
agg = df.groupby('G142', as_index=False).agg({'ND':'count'})

In [57]:
# average declarations per month during the requested period

agg['avg_declarations'] = (agg['ND'] / df.month_year.nunique()).round()
agg = agg[['G142', 'avg_declarations']]

In [58]:
# convert country code to continent name

def assign_continents(x):
    if type(x) == str:
        у = pc.country_alpha2_to_continent_code(x)
        return pc.convert_continent_code_to_continent_name(у)
    return None

df['continent'] = df['G15A'].apply(assign_continents)

In [59]:
# restructuring data for further aggregation

df['incoterms_country'] = df['G202'] + '_' + df['G15']


def concatenate_inn(inn, firm):
    if type(inn) == str and type(firm) == str:
        return 'ИНН:' + inn.strip() + ' | ' + firm.strip()
    elif type(inn) == str and type(firm) != str:
        return 'ИНН:' + inn.strip()
    return firm

df['company_info'] = df.apply(lambda x: concatenate_inn(x.G141, x.FIRM), axis=1)

df['customs_point'] = df['G071'].astype(str) + '_' + df['G30CITY']

In [60]:
# assign late tag to companies with any non-zero number of late shipments

late = df.groupby('G142', as_index=False).agg({'late_release':'mean'})
late['late_tag'] = late['late_release'].apply(lambda x: 'late' if x > 0 else '')
late = late[['G142', 'late_tag']]

In [61]:
# filtering only necessary columns

df = df[['G142', 'G011', 'G143', 'G15', 'G541_INN', 'G33', 'continent', 'incoterms_country', 'company_info', 'customs_point']]

In [62]:
# group by company, concatenating all values in all columns

agg_import = df.fillna('').astype(str).groupby('G142', as_index=False).agg(
    lambda x: re.sub(',+', ', ', ', '.join(x.unique())))

In [63]:
# merge data on late tags and average declaration to main dataframe

agg_import = agg_import.merge(agg, how='left', on='G142')
agg_import = agg_import.merge(late, how='left', on='G142')

In [64]:
agg_import.head(3)

,G142,G011,G143,G15,G541_INN,G33,continent,incoterms_country,company_info,customs_point,avg_declarations,late_tag
0,"5С ГРУПП, ООО",Импорт,"111020, , МОСКВА, БОРОВАЯ, Д.7, СТР.30,","ТАЙВАНЬ (КИТАЙ), УЗБЕКИСТАН, ОБЪЕДИНЕННЫЕ АР...",,8473302008,Asia,"CPT_ТАЙВАНЬ (КИТАЙ), CPT_УЗБЕКИСТАН, CPT_ОБЪ...",ИНН:7722784854 | ОГРН:1127746675320 | Рук.:Нов...,"10005030_ТЕР., 10005030_Г. ХИМКИ",1.0,
1,DEXP LNTERNATIONAL LIMITED,Импорт,", , WANCHAI, RM 12110, 12/F, YF LIFE TOWE...",НИДЕРЛАНДЫ,7716640565,8473302008,Europe,EXW_НИДЕРЛАНДЫ,ИНН:69868473##,10013160_С. АКУЛОВО,0.0,
2,OOO СИТИДИАЙ РАША,Импорт,"117556, ГОРОД, МОСКВА, ЧОНГАРСКИЙ БУЛЬВАР, ...",ПОЛЬША,,8473302008,Europe,DAP_ПОЛЬША,ИНН:7730608563 | ОГРН:1097746188748 | Рук.:Пеш...,10005030_Г. ХИМКИ,0.0,late


In [65]:
writer = pd.ExcelWriter('PythonExport.xlsx')
agg_import.to_excel(writer,'Sheet5')
writer.save()